In [1]:
# import
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_openai import OpenAIEmbeddings
import openai
import pandas as pd
import csv
import re
import faiss
import numpy as np

In [2]:
import json

input_json_path = 'combined_eval.json'

with open(input_json_path, 'r') as json_file:
    data_list = json.load(json_file)

print(data_list[0])

{'Hutsul': 'А тобі, Любка, тра’ мами?', 'Ukrainian': 'А тобі, Любка, потрібна мама?'}


In [3]:
from openai import OpenAI

client = OpenAI(
    api_key='some_key'
)

In [4]:
import os
os.environ["OPENAI_API_KEY"] = 'some_key'

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
def generate_ukrainian_prompt(sentence):
    prompt = f"""Речення літературною українською: "{sentence}".
    
Перепиши речення: "{sentence}" на гуцульський діалект, використовуючи основні особливості і правила гуцульського діалекту.

У відповіді поверни ТІЛЬКИ переписане тобою речення у гуцульському діалекті, без жодних коментарів чи пояснень. Поверни ТІЛЬКИ переписане речення. 
"""

    return prompt

In [7]:
def generate_ukrainian_prompt_to_ukrainian_translate(sentence):
    prompt = f"""Ти експерт у гуцульському діалекті, і твоя задача: переписувати надані речення з гуцульського діалекту на літературну українську.
    
Речення гуцульським діалектом: "{sentence}".
    
Переклади речення: "{sentence}" на літературну українську.

У відповіді поверни ТІЛЬКИ переписане тобою речення на літературну українську, без жодних коментарів чи пояснень. Поверни ТІЛЬКИ перекладене речення. 
"""

    return prompt

In [8]:
def generate_instructions():
    prompt = f"""Ти експерт у гуцульському діалекті, і твоя задача: переписувати надані речення гуцульським діалектом.

Ти застосовуєш такі основні особливості і правила гуцульського діалекту при переписуванні речень гуцульським діалектом:
- Заміна "є" замість "а", "я" (коли наголошене). Приклад: гуцульський - "єк", українською - "як"; гуцульський - "єгода", українською - "ягода"; гуцульський - "єйце", українською - "яйце"; гуцульський - "єрмарок", українською - "ярмарок".
- Заміна "е" або рідше "і" замість "и" (коли наголошене). Приклад: гуцульський - "абес", українською - "абись"; гуцульський - "жето", українською - "жито". 
- Заміна "і" на "и". Приклад: гуцульський - "Иван", українською - "Іван"; гуцульський - "иду", українською - "іду".
- Заміна "я" на "и" або "і": Приклад: гуцульський - "забаривсми-си", українською - "забарився-ся"; гуцульський - "най-си", українською - "най-ся"
- Заміна "ю" на "у". Приклад: гуцульський - "суда", українською - "сюди"; гуцульський - "вівцу", українською - "вівцю".
- "ві-" замість "ви-" у префіксі. Приклад: гуцульський - "вітратити", українською - "витратити".
- Заміна "д" на "ґ". Приклад: гуцульський - "ґівка", українською - "дівка"; гуцульський - "ґедьо", українською - "дідо".
- Заміна "к" на "ть". Приклад: гуцульський - "доньтя", українською - "донька".
- Пом'якшення шиплячих. Приклад: гуцульський - "чьо", українською - "чого"; гуцульський - "чєлєдь", українською - "челядь".
- "д" замість "ґ". Приклад: гуцульський - "ледінь", українською - "леґінь, легінь".
- "к" замість "т" перед "є", "і", іноді в кінці слова. Приклад: гуцульський - "крескєни", українською - "християни"; гуцульський - "кокє", українською - "котя"; гуцульський - "кєшко", українською - "тяжко".
- "ц" замість "ч" і навпаки. Приклад: гуцульський - "ци", українською - "чи"; гуцульський - "чибук", українською - "цибух".
- тверді "с", "ц", "з" замість пом'якшених. Приклад: гуцульський - "шос", українською - "шось"; гуцульський - "дес", українською - "десь"; гуцульський - "хлопец", українською - "хлопець".
- Заміна "и" на "і", і навпаки в множині іменників. Приклад: гуцульський - "ангелі", українською - "ангели"; гуцульський - "Жебівци", українською - "Жебівці".
- "нн", "н" замість "дн", "тн", "лн". Приклад: гуцульський - "перенний", українською - "передний"; гуцульський - "мeнник" або "мeнник", українською - "мелник".
- Іменники основ "-а", "-я" в орудному відмінку мають закінчення "-ов", "-ев": руков «рукою», землев «землею»;

Приклади твого перекладу з літературної української мови на гуцульський діалект:
Літературна українська: За річкою, саме наперед вікон, під лісом тетеріла стрімка стряпава скеля.
Гуцульський діалект: За річьков, наопередь самих викон, пид лісом тетеріла стримка, стряпава скала.

У своїй відповіді ти повертаєш ТІЛЬКИ переписане тобою речення у гуцульському діалекті, без коментарів та різних згадувань. Лише речення написане гуцульським діалектом.
"""

    return prompt

In [9]:
print(generate_instructions())

Ти експерт у гуцульському діалекті, і твоя задача: переписувати надані речення гуцульським діалектом.

Ти застосовуєш такі основні особливості і правила гуцульського діалекту при переписуванні речень гуцульським діалектом:
- Заміна "є" замість "а", "я" (коли наголошене). Приклад: гуцульський - "єк", українською - "як"; гуцульський - "єгода", українською - "ягода"; гуцульський - "єйце", українською - "яйце"; гуцульський - "єрмарок", українською - "ярмарок".
- Заміна "е" або рідше "і" замість "и" (коли наголошене). Приклад: гуцульський - "абес", українською - "абись"; гуцульський - "жето", українською - "жито". 
- Заміна "і" на "и". Приклад: гуцульський - "Иван", українською - "Іван"; гуцульський - "иду", українською - "іду".
- Заміна "я" на "и" або "і": Приклад: гуцульський - "забаривсми-си", українською - "забарився-ся"; гуцульський - "най-си", українською - "най-ся"
- Заміна "ю" на "у". Приклад: гуцульський - "суда", українською - "сюди"; гуцульський - "вівцу", українською - "вівцю".


In [10]:
def chatGPT_api(messages):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature=0.0
    )

    return chat_completion.choices[0].message

def send_llm_request_to_ukrainian_translate(sentence):
    content = generate_ukrainian_prompt_to_ukrainian_translate(sentence)
    messages = [
        {"role": "user", "content": content}
    ]
    synthetic_sentence = chatGPT_api(messages).content

    return synthetic_sentence


def send_llm_request_to_hutsul_translate(sentence, instructions):
    content = generate_ukrainian_prompt(sentence)
    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": content}
    ]
    synthetic_sentence = chatGPT_api(messages).content

    return synthetic_sentence

In [16]:
model_name = "chatGpt-3.5"
output_text_path = model_name+'_preprocessed_eval.txt'
instruction = generate_instructions()

for data_pair in data_list:
    try:
        sentence = data_pair['Ukrainian']
        text = send_llm_request_to_hutsul_translate(sentence, instruction)
        print(f"Ukrainian: {data_pair['Ukrainian']} Hutsul: {text}\n")
        with open(output_text_path, 'a') as output_file:
            output_file.write(f"Ukrainian: {data_pair['Ukrainian']} Hutsul: {text}\n")
            output_file.write(f"!!!\n")

        sentence_hutsul = data_pair['Hutsul']
        text_ukrainian = send_llm_request_to_ukrainian_translate(sentence_hutsul)
        print(f"Hutsul: {data_pair['Hutsul']} Ukrainian: {text_ukrainian}\n")
        with open(output_text_path, 'a') as output_file:
            output_file.write(f"Hutsul: {data_pair['Hutsul']} Ukrainian: {text_ukrainian}\n")
            output_file.write(f"-------------------------------\n")
    except Exception as e:
        print(data_pair)
        print(e)
        print("something went wrong")
        print('------------!---------------')

Ukrainian: А тобі, Любка, потрібна мама? Hutsul: А тоби, Любко, потрібна мама?

Hutsul: А тобі, Любка, тра’ мами? Ukrainian: А тобі, Любко, треба мами?

Ukrainian: Зараз покличу. Hutsul: Цярас покличу.

Hutsul: Зара’ закличу. Ukrainian: Зараз закличу.

Ukrainian: Бо я плоти на болоті ремонтую, а мама на Мазянівці б’є білизну. Hutsul: Бо ї плоти на болоті ремонтую, а мама на Мазянівці б’є білизну.

Hutsul: Бо я ади на Багні лагожу плоти, а мама на Мазянівці б’є белеґу. Ukrainian: Бо я їду на Багні ловити рибу, а мама на Мазянівці миє посуд.

Ukrainian: Можливо він зателефонує назад! Hutsul: Можливо він затилефонує назад!

Hutsul: Мо’ витак най перезвонит! Ukrainian: Мені треба тобі передзвонити!

Ukrainian: А як ти туди: як Австрія, Німеччина? Hutsul: А як ти туди: як Австрія, Німеччина?

Hutsul: А єк ти туда: єк Австрія, Німеччіна? Ukrainian: А ти туди пішов: до Австрії, Німеччини?

Ukrainian: Я у Львові: і тут немає ні Австрії, ні Німеччини! Hutsul: И у Львові: і тут немає ні Австрії,